In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.datasets import make_classification

train = pd.read_csv('/Users/yang/Downloads/train.csv')

Duplicate key in file '/Users/yang/.matplotlib/matplotlibrc' line #2.


In [3]:
DF = train.iloc[:,[1,2,4,5,6,7,9,11]]

In [4]:
# 결측치 비율 확인
DF.isnull().sum()/DF.shape[0]*100

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Fare         0.000000
Embarked     0.224467
dtype: float64

In [5]:
DF.Age = DF.Age.fillna(DF.Age.median())
DF.Embarked = DF.Embarked.fillna(DF.Embarked.mode()[0])
DF.isnull().sum()/DF.shape[0]*100

/Users/yang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Survived    0.0
Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
Embarked    0.0
dtype: float64

In [6]:
DF

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [7]:
# 범주형 변수들을 label encoding / one hot encoding 해주어야함
## 교수님 방식대로 label encoder --> one hot encoder

categ_columns=['Embarked'] # 범주형 변수

def dummy(data,col):
    lab=LabelEncoder() #0~c-1로 클래스 부여
    aa=lab.fit_transform(DF[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [8]:
for column in categ_columns:
    temp_df=dummy(DF,column)

/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
## 변수가 ordinal이면 label encoder / nominal이면 one-hot-encoder

In [10]:
temp_df.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


In [11]:
# 기존 Embarked 변수를 onehotencoding된 변수로 대체
del DF['Embarked']

In [12]:
DF_encoding = pd.concat([DF, temp_df], axis=1)

In [13]:
DF_encoding

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,male,22.0,1,0,7.2500,0.0,0.0,1.0
1,1,1,female,38.0,1,0,71.2833,1.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,1,female,35.0,1,0,53.1000,0.0,0.0,1.0
4,0,3,male,35.0,0,0,8.0500,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,0.0,0.0,1.0
887,1,1,female,19.0,0,0,30.0000,0.0,0.0,1.0
888,0,3,female,28.0,1,2,23.4500,0.0,0.0,1.0
889,1,1,male,26.0,0,0,30.0000,1.0,0.0,0.0


In [14]:
# Sex변수는 Labelencoding만

le = LabelEncoder()
DF_encoding['Sex'] = le.fit_transform(DF_encoding.Sex)

In [15]:
DF_encoding

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0.0,0.0,1.0
1,1,1,0,38.0,1,0,71.2833,1.0,0.0,0.0
2,1,3,0,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,1,0,35.0,1,0,53.1000,0.0,0.0,1.0
4,0,3,1,35.0,0,0,8.0500,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0.0,0.0,1.0
887,1,1,0,19.0,0,0,30.0000,0.0,0.0,1.0
888,0,3,0,28.0,1,2,23.4500,0.0,0.0,1.0
889,1,1,1,26.0,0,0,30.0000,1.0,0.0,0.0


In [16]:
# 전체 data를 training set과 test set으로 split

## DF.iloc[:,0].values  --> values를 붙여주면 array형식으로 바뀜
from sklearn.model_selection import train_test_split
X, y = DF_encoding.iloc[:,1:].values, DF_encoding.iloc[:,0].values

X_train, X_test, y_train,y_test = \
    train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [17]:
# 표준화
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [18]:
# Logistic Regression with L2 or L1 Regularization

from sklearn.linear_model import LogisticRegression
lr2_10 = LogisticRegression(penalty='l2', C=10.0)  # L2 with C(=1/λ)=10
lr2_1 = LogisticRegression(penalty='l2', C=1.0)    # L2 with C(=1/λ)=1
lr2_0_1 = LogisticRegression(penalty='l2', C=0.1)  # L2 with C(=1/λ)=0.1
lr1_10 = LogisticRegression(penalty='l1', C=10.0)  # L1 with C(=1/λ)=10
lr1_1 = LogisticRegression(penalty='l1', C=1.0)    # L1 with C(=1/λ)=1
lr1_0_1 = LogisticRegression(penalty='l1', C=0.1)  # L1 with C(=1/λ)=0.1

In [19]:
# 규제화 방법(L2 or L1)과 규제강도(λ)를 바꿔가며 accuracy score 계산
lr2_10.fit(X_train_std, y_train)
print('Training accuracy with L2 and λ=0.1:', lr2_10.score(X_train_std, y_train))
print('Test accuracy with L2 and λ=0.1:', lr2_10.score(X_test_std, y_test))

lr2_1.fit(X_train_std, y_train)  # warning..
print('Training accuracy with L2 and λ=1:', lr2_1.score(X_train_std, y_train))
print('Test accuracy with L2 and λ=1:', lr2_1.score(X_test_std, y_test))

lr2_0_1.fit(X_train_std, y_train)
print('Training accuracy with L2 and λ=10:', lr2_0_1.score(X_train_std, y_train))
print('Test accuracy with L2 and λ=10:', lr2_0_1.score(X_test_std, y_test))

lr1_10.fit(X_train_std, y_train)
print('Training accuracy with L1 and λ=0.1:', lr1_10.score(X_train_std, y_train))
print('Test accuracy with L1 and λ=0.1:', lr1_10.score(X_test_std, y_test))

lr1_1.fit(X_train_std, y_train)
print('Training accuracy with L1 and λ=1:', lr1_1.score(X_train_std, y_train))
print('Test accuracy with L1 and λ=1:', lr1_1.score(X_test_std, y_test))

lr1_0_1.fit(X_train_std, y_train)
print('Training accuracy with L1 and λ=10:', lr1_0_1.score(X_train_std, y_train))
print('Test accuracy with L1 and λ=10:', lr1_0_1.score(X_test_std, y_test))

Training accuracy with L2 and λ=0.1: 0.8154093097913323
Test accuracy with L2 and λ=0.1: 0.7686567164179104
Training accuracy with L2 and λ=1: 0.8154093097913323
Test accuracy with L2 and λ=1: 0.7686567164179104
Training accuracy with L2 and λ=10: 0.8138041733547352
Test accuracy with L2 and λ=10: 0.7798507462686567
Training accuracy with L1 and λ=0.1: 0.8154093097913323
Test accuracy with L1 and λ=0.1: 0.7686567164179104
Training accuracy with L1 and λ=1: 0.8154093097913323
Test accuracy with L1 and λ=1: 0.7686567164179104
Training accuracy with L1 and λ=10: 0.8025682182985554
Test accuracy with L1 and λ=10: 0.7835820895522388


/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be ch

In [20]:
## Grid Search



In [21]:
## 결론: logistic돌리려면 데이터가 array형식 이어야 한다. 이는 data.iloc[].values를 통해 추출 가능

**5장 LDA & QDA & **

In [22]:
#### 타이타닉 data에 대한 LDA 적합 ####
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

cld=LinearDiscriminantAnalysis(store_covariance=True)
cld.fit(X_train, y_train) # LDA 적합

/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=True, tol=0.0001)

In [23]:
y_train_pred=cld.predict(X_train)
y_test_pred=cld.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, y_train_pred)) # train data에 대한 accuracy
print(accuracy_score(y_test, y_test_pred)) # test data에 대한 accuracy

0.8089887640449438
0.7873134328358209


In [26]:
# 분류 결과
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))  # 각 행은 survived=0, survived=1

[[143  22]
 [ 35  68]]


In [27]:
#### 타이타닉 data에 대한 QDA적합 ####
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

cqd=QuadraticDiscriminantAnalysis(store_covariance=True)
cqd.fit(X_train, y_train) # QDA 적합

/Users/yang/opt/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0, store_covariance=True,
                              tol=0.0001)

In [28]:
y_train_pred=cqd.predict(X_train)
y_test_pred=cqd.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, y_train_pred)) # train data에 대한 accuracy
print(accuracy_score(y_test, y_test_pred)) # test data에 대한 accuracy

0.7158908507223114
0.6865671641791045


In [30]:
# 분류 결과
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_test_pred))

[[103  62]
 [ 22  81]]


**6장. Classification Tree**